In [1]:
!pip install pandas

  Using cached pandas-2.3.3-cp314-cp314-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.3.3-cp314-cp314-macosx_11_0_arm64.whl (10.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 27.8 MB/s  0:00:00m0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Probleme par commune traitement pour visualisation

In [ ]:
import pandas as pd
import re

# 1. Chargement du fichier
input_file = "problemes_par_communes.csv"
df = pd.read_csv(input_file)

# 2. Dictionnaire de correspondance INSEE -> Nom (Métropole de Lyon)
insee_map = {
    '69003': 'Albigny-sur-Saône', '69029': 'Bron', '69033': 'Cailloux-sur-Fontaines',
    '69034': 'Caluire-et-Cuire', '69040': 'Champagne-au-Mont-d\'Or', '69044': 'Charbonnières-les-Bains',
    '69046': 'Charly', '69271': 'Chassieu', '69063': 'Collonges-au-Mont-d\'Or',
    '69273': 'Corbas', '69068': 'Couzon-au-Mont-d\'Or', '69069': 'Craponne',
    '69071': 'Curis-au-Mont-d\'Or', '69072': 'Dardilly', '69275': 'Décines-Charpieu',
    '69081': 'Écully', '69276': 'Feyzin', '69085': 'Fleurieu-sur-Saône',
    '69087': 'Fontaines-Saint-Martin', '69088': 'Fontaines-sur-Saône', '69089': 'Francheville',
    '69278': 'Genay', '69091': 'Givors', '69096': 'Grigny', '69100': 'Irigny',
    '69279': 'Jonage', '69116': 'Limonest', '69117': 'Lissieu', '69123': 'Lyon',
    '69127': 'Marcy-l\'Étoile', '69282': 'Meyzieu', '69283': 'Mions', '69284': 'Montanay',
    '69142': 'La Mulatière', '69143': 'Neuville-sur-Saône', '69149': 'Oullins-Pierre-Bénite',
    '69152': 'Pierre-Bénite', '69153': 'Poleymieux-au-Mont-d\'Or', '69163': 'Quincieux',
    '69286': 'Rillieux-la-Pape', '69168': 'Rochetaillée-sur-Saône', '69191': 'Saint-Cyr-au-Mont-d\'Or',
    '69194': 'Saint-Didier-au-Mont-d\'Or', '69199': 'Saint-Fons', '69204': 'Saint-Genis-Laval',
    '69205': 'Saint-Genis-les-Ollières', '69207': 'Saint-Germain-au-Mont-d\'Or', '69290': 'Saint-Priest',
    '69233': 'Saint-Romain-au-Mont-d\'Or', '69202': 'Sainte-Foy-lès-Lyon', '69292': 'Sathonay-Camp',
    '69293': 'Sathonay-Village', '69296': 'Solaize', '69244': 'Tassin-la-Demi-Lune',
    '69250': 'La Tour-de-Salvagny', '69256': 'Vaulx-en-Velin', '69259': 'Vénissieux',
    '69260': 'Vernaison', '69266': 'Villeurbanne'
}

# 3. Application du mapping
# On s'assure que la colonne INSEE est bien en texte
df['INSEE'] = df['INSEE'].astype(str)
df['Nom_Commune'] = df['INSEE'].map(insee_map).fillna(df['INSEE'])

# 4. Transformation des Top 10 problèmes
processed_rows = []

for idx, row in df.iterrows():
    raw_string = row['Top_10_Problemes']
    if pd.isna(raw_string):
        continue
        
    # On découpe la chaîne par le séparateur " | "
    problems = raw_string.split(' | ')
    
    for rank, item in enumerate(problems, 1):
        # On extrait le nom et le chiffre entre parenthèses via Regex
        match = re.search(r'^(.*)\s\((\d+)\)$', item.strip())
        if match:
            prob_name = match.group(1).strip()
            prob_count = int(match.group(2))
            
            processed_rows.append({
                'Nom_Commune': row['Nom_Commune'],
                'INSEE': row['INSEE'],
                'Total_Reponses_Commune': row['Réponses'],
                'Note_Securite': row['Sécurité_Moy'],
                'Probleme': prob_name,
                'Nombre_Signalements': prob_count,
                'Classement': rank
            })

# Création du nouveau DataFrame
df_dashboard = pd.DataFrame(processed_rows)

# 5. Sauvegarde
df_dashboard.to_csv('donnees_dashboard_communes.csv', index=False, encoding='utf-8-sig')
print("Transformation terminée ! Fichier 'donnees_dashboard_communes.csv' créé.")
print(df_dashboard.head())

Transformation terminée ! Fichier 'donnees_dashboard_communes.csv' créé.
  Nom_Commune  INSEE  Total_Reponses_Commune  Note_Securite  \
0        Lyon  69123                    6167           3.96   
1        Lyon  69123                    6167           3.96   
2        Lyon  69123                    6167           3.96   
3        Lyon  69123                    6167           3.96   
4        Lyon  69123                    6167           3.96   

                   Probleme  Nombre_Signalements  Classement  
0  Infrastructure manquante                 5064           1  
1         Vitesse excessive                 4915           2  
2         Violence routière                 3862           3  
3           Conflit piétons                 3435           4  
4      Stationnement gênant                 2725           5  
